# Full Workflow: Advanced Price Engine

This notebook demonstrates the complete workflow of the Advanced Price Engine, from simulating data to getting a profit-optimizing price recommendation via the API for multiple models.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
sns.set_theme(style="whitegrid")

## 1. Interact with the Live API

Let's assume the API is running (e.g., via `docker-compose up`). We can query its endpoints to get model details and optimization results.

In [ ]:
BASE_URL = "http://localhost:8000"

try:
    # First, let's see which models are available
    response = requests.get(f"{BASE_URL}/models")
    response.raise_for_status()
    available_models = response.json()
    print("--- Available Models and Elasticities ---")
    print(json.dumps(available_models, indent=2))
except requests.exceptions.ConnectionError as e:
    print(f"Could not connect to the API at {BASE_URL}. Please ensure it is running.")

## 2. Get Profit Optimization Results

Now, let's ask the API to find the optimal price for a product with a marginal cost of $8, searching for a price between $15 and $30.

In [ ]:
OPTIMIZE_URL = f"{BASE_URL}/optimize-profit"
MARGINAL_COST = 8.0
PRICE_RANGE = (15.0, 30.0)

if 'available_models' in locals():
    for model_name in available_models:
        params = {
            'model_name': model_name,
            'marginal_cost': MARGINAL_COST,
            'min_price': PRICE_RANGE[0],
            'max_price': PRICE_RANGE[1]
        }
        try:
            response = requests.post(OPTIMIZE_URL, params=params)
            response.raise_for_status() 
            recommendation = response.json()
            print(f"\n--- Optimization Result from '{model_name.upper()}' Model ---")
            print(json.dumps(recommendation, indent=2))
        except requests.exceptions.HTTPError as e:
            print(f"API returned an error for model {model_name}: {e.response.status_code} {e.response.text}")